Code to build the term document matrix. What do I doooo?

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import cleaner

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df_prods = pd.read_csv('../data/product_descriptions.csv')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords

import numpy as np
vect = CountVectorizer(min_df=1, stop_words=stopwords.words('english'))
X_vect = vect.fit_transform(df_prods['product_description'])


In [27]:
tfidf = TfidfVectorizer(min_df=1, stop_words=stopwords.words('english'))
X_tfidf = tfidf.fit_transform(df_prods['product_description'])

In [5]:
# How to convert a sentance to a vector
# gives back a 1 x p vector and counts the occurences
print vect.transform(['common angles'])

  (0, 21617)	1
  (0, 57044)	1


In [6]:
print tfidf.shape

(124428, 268737)


# Comapre with the search term

In [9]:
df_train = pd.read_csv('../data/train_sample.csv')

In [18]:
vect.transform(df_train.head(n=1)['search_term'])

<1x268737 sparse matrix of type '<type 'numpy.int64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

In [39]:

row_num = 2
row = df_train.loc[row_num]
print row.relevance
print row['search_term']
print row['product_description']

3.0
chair molding
Complete your bathroom with this matching accessory set. This set includes a single towel rod and a double robe hook. Constructed with lead-free brass, this set is finished with beautiful premium chrome plating.2 hooks to maximize storage spaceSingle towel rod can hold up to 6 lbs. constructed with lead-free brass, ensuring strength and durabilityQuality control approved in Canada and re-inspected prior to shipping your order; premium chrome plating to suit any home decorMounting hardware included; solid bulky brass feelConcealed screw installation for a clean lookDesigned for easy installation


In [59]:

row_num = df_prods[df_prods['product_uid'] == row['product_uid']].index
cosine_similarity(X_vect[row_num],vect.transform([row['search_term']]))

array([[ 0.36247326]])

In [58]:

row_num = df_prods[df_prods['product_uid'] == row['product_uid']].index
cosine_similarity(X_tfidf[row_num],tfidf.transform([row['search_term']]))

array([[ 0.35376704]])